In [300]:
#importing elastic search 
from elasticsearch import Elasticsearch

In [301]:
#connecting to the server
es= Elasticsearch("https://localhost:9200",
                  basic_auth=('elastic', 'UXjHr=I*dSOPFsfM3rIJ'),
                  verify_certs=False)
es.ping()
                  

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


True

In [302]:
import pandas as pd
import numpy as np

**Preprocessing and Cleaning the data**

In [303]:
#reading dataset into dataframe
movies = pd.read_csv("imdb_100.csv")


columns_to_drop = ['Poster_Link', 'Meta_score', 'No_of_Votes','Gross']
movies.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [304]:
movies.drop(columns=columns_to_drop, inplace=True)

movies.head()

#combining the all the actors column into cast
movies['Cast'] = movies[['Star1', 'Star2', 'Star3', 'Star4']].apply(lambda x: ' , '.join(x[x.notnull()]), axis=1)

movies['Cast'] = movies['Star1'] +", "+ movies['Star2']+", "+ movies['Star3']+", "+ movies['Star4']

movies.drop(columns=['Star1','Star2','Star3','Star4'],inplace=True)

In [305]:
#seeing the top 5 rows of dataset
movies.head()

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Director,Cast
0,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi..."
1,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke..."
2,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M..."
3,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,Francis Ford Coppola,"Al Pacino, Robert De Niro, Robert Duvall, Dian..."
4,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,Sidney Lumet,"Henry Fonda, Lee J. Cobb, Martin Balsam, John ..."


In [306]:
movies.info

<bound method DataFrame.info of                  Series_Title Released_Year Certificate  Runtime  \
0    The Shawshank Redemption          1994           A  142 min   
1               The Godfather          1972           A  175 min   
2             The Dark Knight          2008          UA  152 min   
3      The Godfather: Part II          1974           A  202 min   
4                12 Angry Men          1957           U   96 min   
..                        ...           ...         ...      ...   
995    Breakfast at Tiffany's          1961           A  115 min   
996                     Giant          1956           G  201 min   
997     From Here to Eternity          1953      Passed  118 min   
998                  Lifeboat          1944         NaN   97 min   
999              The 39 Steps          1935         NaN   86 min   

                        Genre  IMDB_Rating  \
0                       Drama          9.3   
1                Crime, Drama          9.2   
2        Acti

In [307]:
# checking for null values
movies.isna().sum()

Series_Title       0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Director           0
Cast               0
dtype: int64

In [308]:
#dropping the "certifcate" and "runtime" columns
movies.drop(columns=['Certificate'],inplace=True)

In [309]:
#check for duplicated rows
movies.loc[movies.duplicated()]

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Overview,Director,Cast


In [310]:
#combining for overview and genre columns for embedding
movies['Column_for_embedding'] = movies['Series_Title'] + ' ' + movies['Genre'] + ' ' + movies['Cast'] + ' ' + movies['Overview']+ ' ' + movies['Director'] + ' ' + movies['Released_Year']
movies['Column_for_embedding'] = movies['Column_for_embedding'].str.lower()  # Convert to lowercase

movies.head()

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Overview,Director,Cast,Column_for_embedding
0,The Shawshank Redemption,1994,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...","the shawshank redemption drama tim robbins, mo..."
1,The Godfather,1972,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...","the godfather crime, drama marlon brando, al p..."
2,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...","the dark knight action, crime, drama christian..."
3,The Godfather: Part II,1974,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,Francis Ford Coppola,"Al Pacino, Robert De Niro, Robert Duvall, Dian...","the godfather: part ii crime, drama al pacino,..."
4,12 Angry Men,1957,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,Sidney Lumet,"Henry Fonda, Lee J. Cobb, Martin Balsam, John ...","12 angry men crime, drama henry fonda, lee j. ..."


In [311]:

#null values sum in movies dataset
movies.isna().sum()

Series_Title            0
Released_Year           0
Runtime                 0
Genre                   0
IMDB_Rating             0
Overview                0
Director                0
Cast                    0
Column_for_embedding    0
dtype: int64

In [312]:
# checking for duplicate rows according to series title column
duplicate_rows = movies[movies.duplicated(subset=['Series_Title'])]
print(duplicate_rows)

    Series_Title Released_Year  Runtime                  Genre  IMDB_Rating  \
136     Drishyam          2015  163 min  Crime, Drama, Mystery          8.2   

                                              Overview         Director  \
136  Desperate measures are taken by a man who trie...  Nishikant Kamat   

                                             Cast  \
136  Ajay Devgn, Shriya Saran, Tabu, Rajat Kapoor   

                                  Column_for_embedding  
136  drishyam crime, drama, mystery ajay devgn, shr...  


In [313]:
#dropping the duplicated rows
movies=movies.drop_duplicates(subset=['Series_Title'])

In [314]:
#resetting the index after dropping duplicate rows
movies = movies.reset_index(drop=True)

In [315]:
#finding the shape of dataset
movies.shape

(999, 9)

In [316]:
# checking for null values in each column
movies.isna().value_counts()

Series_Title  Released_Year  Runtime  Genre  IMDB_Rating  Overview  Director  Cast   Column_for_embedding
False         False          False    False  False        False     False     False  False                   999
Name: count, dtype: int64

In [317]:
#filling na with None
movies.fillna("None", inplace=True)

In [318]:
# importing sentence transformers using all-mpnet-base-v2 model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [319]:
#embedding the column to create vector embeddings using the model
movies['Embedded_vectors']= movies['Column_for_embedding'].apply(lambda x: model.encode(x))

In [320]:
#checking for embeddings
movies.head()

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Overview,Director,Cast,Column_for_embedding,Embedded_vectors
0,The Shawshank Redemption,1994,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...","the shawshank redemption drama tim robbins, mo...","[0.026611099, 0.045497987, -0.0020557905, -0.0..."
1,The Godfather,1972,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...","the godfather crime, drama marlon brando, al p...","[-0.02169593, 0.0731786, -0.0008414599, 0.0579..."
2,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...","the dark knight action, crime, drama christian...","[0.010422537, 0.014041155, -0.015102025, -0.02..."
3,The Godfather: Part II,1974,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,Francis Ford Coppola,"Al Pacino, Robert De Niro, Robert Duvall, Dian...","the godfather: part ii crime, drama al pacino,...","[-0.019417109, 0.1087455, -0.010890105, 0.0335..."
4,12 Angry Men,1957,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,Sidney Lumet,"Henry Fonda, Lee J. Cobb, Martin Balsam, John ...","12 angry men crime, drama henry fonda, lee j. ...","[0.016572058, 0.0092325155, 0.025293067, 0.025..."


In [321]:
#checking to see if the connection is open
es.ping()

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


True

**Inserting indexMapping into elastic search**

In [322]:
#importing indexmapping file and using indexMapping variable 
# created an index to the index mappings
from indexMapping import indexMapping
es.indices.create(index="all_m10s",mappings=indexMapping)

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_m10s'})

In [323]:
#converting the movies to a list of dictionary 
records_list=movies.to_dict("records")

In [324]:
#accessing the first record of the list of dictionaries created above
records_list[0]

{'Series_Title': 'The Shawshank Redemption',
 'Released_Year': '1994',
 'Runtime': '142 min',
 'Genre': 'Drama',
 'IMDB_Rating': 9.3,
 'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
 'Director': 'Frank Darabont',
 'Cast': 'Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler',
 'Column_for_embedding': 'the shawshank redemption drama tim robbins, morgan freeman, bob gunton, william sadler two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency. frank darabont 1994',
 'Embedded_vectors': array([ 2.66110990e-02,  4.54979874e-02, -2.05579051e-03, -9.29718092e-03,
        -4.64870548e-03,  2.46590059e-02,  6.36774721e-03, -3.04534170e-03,
         5.40904514e-02, -1.42883956e-02, -2.40939646e-03,  3.92413810e-02,
         3.62652540e-02,  1.48451654e-02, -9.84468590e-03, -4.26253863e-02,
        -2.03013755e-02,  3.29834260e-02,  5.70372678e

In [325]:
#pinging the server to check for connection
es.ping()

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


True

In [329]:
#inserting each record into the index created
for record in records_list:
    try:
        es.index(index="all_m10s", document=record)
    except Exception as e:
        print(e)

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/adv

BadRequestError(400, 'document_parsing_exception', "[1:45] failed to parse field [Released_Year] of type [long] in document with id 'Z-iltY4BEjlFEleAlP-1'. Preview of field's value: 'PG'")


/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/adv

In [330]:
#checking for the count of records inserted in the server
es.count(index="all_m10s")

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'count': 998, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

**Querying the database**

In [332]:
#testing for semantic search on a query
input_keyword = "comedy"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "Embedded_vectors",
    "query_vector" : vector_of_input_keyword,
    "k" : 3,
    "num_candidates" : 998, 
}

res = es.knn_search(index="all_m10s", knn=query , source=["Series_Title","Overview","Cast","Genre","Released_Year","Runtime"])
res["hits"]["hits"]

/Users/shivani/Desktop/Machine learning/semantic/venv2/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/var/folders/_b/sx37wgxj4xv1vk7hfznl8lz40000gn/T/ipykernel_60125/254546636.py:12: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_m10s", knn=query , source=["Series_Title","Overview","Cast","Genre","Released_Year","Runtime"])


[{'_index': 'all_m10s',
  '_id': 'builtY4BEjlFEleAY_x1',
  '_score': 0.71366644,
  '_source': {'Series_Title': 'Soul',
   'Released_Year': '2020',
   'Runtime': '100 min',
   'Genre': 'Animation, Adventure, Comedy',
   'Overview': 'After landing the gig of a lifetime, a New York jazz pianist suddenly finds himself trapped in a strange land between Earth and the afterlife.',
   'Cast': 'Kemp Powers, Jamie Foxx, Tina Fey, Graham Norton'}},
 {'_index': 'all_m10s',
  '_id': 'F-iltY4BEjlFEleAf_5T',
  '_score': 0.7136155,
  '_source': {'Series_Title': 'Adams æbler',
   'Released_Year': '2005',
   'Runtime': '94 min',
   'Genre': 'Comedy, Crime, Drama',
   'Overview': 'A neo-nazi sentenced to community service at a church clashes with the blindly devotional priest.',
   'Cast': 'Ulrich Thomsen, Mads Mikkelsen, Nicolas Bro, Paprika Steen'}},
 {'_index': 'all_m10s',
  '_id': 'wuiltY4BEjlFEleAif7c',
  '_score': 0.7062081,
  '_source': {'Series_Title': 'Happiness',
   'Released_Year': '1998',
   